# QIIME 2 Tutorial: Read Processing

This notebook contains materials adapted from a google colab notebook created by the QIIME2 team: https://colab.research.google.com/github/bokulich-lab/uzh-microbiome-tutorial/blob/main/01_read_processing.ipynb; all source code is licensed under the Apache License 2.0.

Save your own local copy of this notebook by using `File > Save a copy in Drive`. At some point you may be prompted to trust the notebook. Trust me, it is safe 🤞

**Disclaimer:**

The Google Colab notebook environment will interpret any command as Python code by default. If we want to run bash commands we will have to prefix them by `!`. So any command you see with a leading `!` is a bash command and if you wanted to run it in your terminal you would omit the leading `!`. For example, if in the Colab notebook you ran `!wget` you would just run `wget` in your terminal.

In this notebook we use the `!` prefix because we run all QIIME 2 commands using the [`q2cli`](https://github.com/qiime2/q2cli/) (QIIME 2 command-line interface). However, QIIME 2 also has a python API and a Galaxy interface. You can learn more about these and other QIIME 2 interfaces at https://qiime2.org/.

Some steps are time-comsuming and the entire notebook may take up to 30-60 minutes, so run the entire notebook now and we will inspect the commands and results as we work through as a class.

## Part 1: Setup

Lets start with the setup. We will import materials from a github repository and go into the materials folder. We will the run a script created by the qiime2 team to install qiime2 and all the plugins. Afterwards we will import the necessary python libraries.

In [1]:
! git clone https://github.com/zajacn/QIIME2_Read_Preprocessing_FGCZ.git materials
! mkdir /content/prefetch_cache

Cloning into 'materials'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 80 (delta 20), reused 50 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (80/80), 19.65 MiB | 16.15 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [2]:
%cd materials

/content/materials


Now we are ready to set up our environment. This will take about 10 minutes.
**Note:** This setup is only relevant for Google Colaboratory and will not work on your local machine. Please follow the [official installation instructions](https://docs.qiime2.org/2023.9/install/) for that. This setup is taken from the setup prepared by bokulich lab.

In [3]:
%run setup_qiime2

[09:32:46] 🐍 Downloading miniforge...                                                           ]8;id=846552;file:///content/materials/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=473477;file:///content/materials/setup_qiime2.py#67\67]8;;\

[09:32:47] 🐍 Done.                                                                              ]8;id=459361;file:///content/materials/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=516023;file:///content/materials/setup_qiime2.py#73\73]8;;\

           🐍 Installing miniforge...                                                            ]8;id=537656;file:///content/materials/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=710448;file:///content/materials/setup_qiime2.py#67\67]8;;\

[09:33:02] 🐍 Installed miniforge to `/usr/local`.                                               ]8;id=620897;file:///content/materials/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=342984;file:///content/materials/setup_qiime2.py#73\73]8;;\

           🔍 Downloading Qiime 2 package list...                                                ]8;id=329530;file:///content/materials/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=112602;file:///content/materials/setup_qiime2.py#67\67]8;;\

...

[09:38:38] 🌲 Done.                                                                              ]8;id=360633;file:///content/materials/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=701714;file:///content/materials/setup_qiime2.py#73\73]8;;\

           📊 Checking that Qiime 2 command line works...                                        ]8;id=550546;file:///content/materials/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=631252;file:///content/materials/setup_qiime2.py#67\67]8;;\

[09:38:52] 📊 Qiime 2 command line looks good 🎉                                                 ]8;id=472929;file:///content/materials/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=574579;file:///content/materials/setup_qiime2.py#73\73]8;;\

           🧹 Cleaned up unneeded files.                                                         ]8;id=897873;file:///content/materials/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=825814;file:///content/materials/setup_qiime2.py#62\62]8;;\

           Everything is A-OK. You can start using Qiime 2 now 👍                               ]8;id=582915;file:///content/materials/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=510879;file:///content/materials/setup_qiime2.py#149\149]8;;\

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns

## Part 2: Import the data.

First what we need to do is convert the fastq sequenced reads into a single object in a qiime2 format (with a qza extension). In order to do that you need a manifest file (manifest.tsv) which provides qiime2 with the absolute path to all the reads. You also need to specify the type of the data and the input format.

Lets check first what are the possible input formats. See the following link for more information: https://docs.qiime2.org/2024.2/tutorials/importing/


In [7]:
!qiime tools list-formats --importable

AlignedDNAFASTAFormat
	No description

AlignedDNASequencesDirectoryFormat
	No description

AlignedProteinFASTAFormat
	No description

AlignedProteinSequencesDirectoryFormat
	No description

AlignedRNAFASTAFormat
	No description

AlignedRNASequencesDirectoryFormat
	No description

AlphaDiversityDirectoryFormat
	No description

AlphaDiversityFormat
	No description

ArtificialGroupingDirectoryFormat
	No description

ArtificialGroupingFormat
	No description

BIOMV100DirFmt
	No description

BIOMV100Format
	No description

BIOMV210DirFmt
	No description

BIOMV210Format
	No description

BLAST6DirectoryFormat
	No description

BLAST6Format
	No description

BLASTDBDirFmtV5
	No description

BooleanSeriesDirectoryFormat
	No description

BooleanSeriesFormat
	No description

Bowtie2IndexDirFmt
	No description

CasavaOneEightLanelessPerSampleDirFmt
	No description

CasavaOneEightSingleLanePerSampleDirFmt
	No description

DADA2StatsDirFmt
	No description

DADA2StatsFormat
	No description

DNAFASTAForm

As you can see you can import any type of data. If you want to start running qiime2 from any type of data you have previously analysed with another tools, you can also do that. Just find the right format to import. Ok lets run the import then.

In [8]:
!qiime tools import --type SampleData[SequencesWithQuality] \
                    --input-path data/manifest.tsv \
                    --output-path data/demux_seqs.qza \
                    --input-format SingleEndFastqManifestPhred33V2

Imported data/manifest.tsv as SingleEndFastqManifestPhred33V2 to data/demux_seqs.qza


In [9]:
!qiime demux summarize --i-data data/demux_seqs.qza \
                       --o-visualization demux_seqs.qzv

Saved Visualization to: demux_seqs.qzv


Now have a look at the demux_seqs.qzv file. How do you do that? You have to go to the folder icon: on the left (just underneath 🔑). Find the demux_seqs.qzv file and download it onto your computer. Then go to [view.qiime2.org](https://view.qiime2.org) and upload that file there. This will produce a report.


Question: How should you trim your reads?

## Part 3. Denoise amplicon sequence variants.

Now that you have imported your reads into qiime you want to construct a feature table. Feature table is a type of artifact accepted by many QIIME 2 plugins/actions and used in many downstream analyses.

There are several ways to construct a feature table in QIIME 2. The major choice to make while working with sequencing data is between ASVs and OTUs. ASVs - clustering the reads by 99%-100% similarity, OTUs - clustering the reads by 97% similarity. Below you will see how to perform denoising of sequences to produce a table of ASVs.

DADA2: Amplicon Sequence Variants
There exist several tools one can use for denoising of NGS reads. Here, we will use DADA2 to create a feature table of ASVs. DADA2 builds an error model which can identify differences between sequences, filters out noisy sequences and generates a feature table with error-corrected sequences.

To denoise the single-end reads we execute the cell below, specifying some additional parameters/outputs:

*   p-trunc-len - we will truncate the reads to 135 bp (sequences shorter than this will be removed automatically)
*   p-n-threads - if we have more than 1 CPU available, we can specify the number here to make the processing faster
*   output-dir:
  *   o-table - this will be our ASVs feature table
  *   o-representative-sequences - this will be a list of all the denoised features (DNA sequences)
  *   o-denoising-stats - this will be some stats from the denoising process

In [10]:
!qiime dada2 denoise-single --i-demultiplexed-seqs data/demux_seqs.qza  \
                            --p-trunc-len 135   \
                            --o-table table.qza   \
                            --o-representative-sequences dada2_rep_set.qza   \
                            --o-denoising-stats dada2_denoising_stats.qza

Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: dada2_denoising_stats.qza


In [11]:
# Optional
! qiime metadata tabulate \
    --m-input-file dada2_denoising_stats.qza \
    --o-visualization dada2_denoising_stats.qzv

Saved Visualization to: dada2_denoising_stats.qzv


In [12]:
! qiime feature-table summarize \
    --i-table table.qza \
    --m-sample-metadata-file data/sample_metadata.tsv \
    --o-visualization table.qzv

Saved Visualization to: table.qzv


In [13]:
# Optional
! qiime feature-table tabulate-seqs \
    --i-data dada2_rep_set.qza \
    --o-visualization representative_sequences.qzv

Saved Visualization to: representative_sequences.qzv


After you have run all of these commands, have a look at the reports. As per usual, download .qzv files and upload them to view.qiime2.org.


Questions:
 * What is the percentage or reads that remains after all the filtering?

## Part 4. Generate a phylogenetic tree.

We need to now generate a phylogenetic tree for diversity metrics. A tree is usually generate in a newick format and you can view that file using [iTOL](https://itol.embl.de/upload.cgi) or [phylo.io](https://beta.phylo.io/viewer/).

In [14]:
! qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences dada2_rep_set.qza \
    --output-dir phylogeny

Saved FeatureData[AlignedSequence] to: phylogeny/alignment.qza
Saved FeatureData[AlignedSequence] to: phylogeny/masked_alignment.qza
Saved Phylogeny[Unrooted] to: phylogeny/tree.qza
Saved Phylogeny[Rooted] to: phylogeny/rooted_tree.qza


### Lets estimate some diversity metrics

First lets generate all diversity metrics. This is usually done by subsampling all samples to the same depth. If you have a huge variability in the depth of your samples this will cause loss of information from the samples that are have higher sequencing depht then the rest.

Afterwards we measure the significance in faith PD alpha diversity across all the samples.

And the finally we will create an Emperor plot (3D PCA) plot


In [15]:
! qiime diversity core-metrics-phylogenetic \
    --i-phylogeny phylogeny/rooted_tree.qza \
    --i-table table.qza \
    --p-sampling-depth 1100 \
    --m-metadata-file data/sample_metadata.tsv \
    --output-dir core-metrics-results

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/bray_curtis_pcoa_re

In [16]:
! qiime diversity alpha-group-significance \
    --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --o-visualization core-metrics-results/faith_pd_group_significance.qzv

Saved Visualization to: core-metrics-results/faith_pd_group_significance.qzv


In [17]:
! qiime emperor plot \
    --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --o-visualization core-metrics-results/bray_curtis_pcoa.qzv

Saved Visualization to: core-metrics-results/bray_curtis_pcoa.qzv


Download the PCoAa and have a look at the clustering of samples. Do you identify anything unusual?

# Part 5: Classify sequences by taxonomy

There are several ways to classify your sequences into bacterial species. One of them is to use consensus assignment based on e.g. BLAST search of a sequence against a database of known sequences. Another one is using a machine learning classifier trained on a reference database to recognize corresponding bacterial species. We will use a pretrained classifier to identify bacterial species present in our samples.

We can use the `classify-sklearn` action from the feature-classifier plugin to do that. This step will require the `FeatureData[Sequence]` artifact (containing our ASVs) that we generated previously and a pre-trained taxonomic classifier. The classifier is based on SILVA database. There are many different databases you can use. The most often used ones are [SILVA](https://www.arb-silva.de/) and [Greengenes](https://greengenes.secondgenome.com/)

In [18]:
! wget https://data.qiime2.org/2023.9/common/gg-13-8-99-515-806-nb-weighted-classifier.qza

--2024-12-03 10:50:39--  https://data.qiime2.org/2023.9/common/gg-13-8-99-515-806-nb-weighted-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2023.9/common/gg-13-8-99-515-806-nb-weighted-classifier.qza [following]
--2024-12-03 10:50:40--  https://s3-us-west-2.amazonaws.com/qiime2-data/2023.9/common/gg-13-8-99-515-806-nb-weighted-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.153.48, 52.92.148.112, 52.92.234.72, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.153.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28738550 (27M) [binary/octet-stream]
Saving to: ‘gg-13-8-99-515-806-nb-weighted-classifier.qza’

gg-13-8-99-515-806- 100%[===================>]  27.41M  41.6MB/s    in

In [19]:
! qiime feature-classifier classify-sklearn \
    --i-reads dada2_rep_set.qza \
    --i-classifier gg-13-8-99-515-806-nb-weighted-classifier.qza \
    --p-n-jobs 2 \
    --output-dir taxonomy

Saved FeatureData[Taxonomy] to: taxonomy/classification.qza


In [20]:
! qiime metadata tabulate \
    --m-input-file taxonomy/classification.qza \
    --o-visualization taxonomy/classification.qzv

Saved Visualization to: taxonomy/classification.qzv


In [21]:
! qiime taxa barplot \
    --i-table table.qza \
    --i-taxonomy taxonomy/classification.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --o-visualization taxonomy/taxa_barplot.qzv

Saved Visualization to: taxonomy/taxa_barplot.qzv


Download the taxa_barplot visualisation. Which genus do you identify as differentially abundant between gut and the two hands?

## Optional section: Understand differentially abundant features

This section may be omitted for time, but provides an interesting mechanistic view of microbiome interactions.

Differential abundance in QIIME2 can be measured with various different metrics. One of them is ANCOMBC2: https://www.nature.com/articles/s41592-023-02092-7

In [22]:
! mkdir diff_abund

! qiime taxa collapse \
    --i-table table.qza \
    --i-taxonomy taxonomy/classification.qza \
    --p-level 6 \
    --o-collapsed-table diff_abund/table_l6.qza

Saved FeatureTable[Frequency] to: diff_abund/table_l6.qza


In [23]:
! qiime composition add-pseudocount \
    --i-table diff_abund/table_l6.qza \
    --o-composition-table diff_abund/comp_table_l6.qza

Saved FeatureTable[Composition] to: diff_abund/comp_table_l6.qza


In [24]:
! qiime feature-table filter-samples \
    --i-table diff_abund/comp_table_l6.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --p-where "[body-site]='gut'" \
    --o-filtered-table diff_abund/comp_gut_table_l6.qza

Saved FeatureTable[Composition] to: diff_abund/comp_gut_table_l6.qza


In [25]:
! qiime composition ancom \
    --i-table diff_abund/comp_gut_table_l6.qza \
    --m-metadata-file data/sample_metadata.tsv \
    --m-metadata-column subject \
    --o-visualization diff_abund/ancom_gut_subject_l6.qzv

Saved Visualization to: diff_abund/ancom_gut_subject_l6.qzv


Download the visualisation (ancom_gut_subject_l6.qzv). How many features are differentially abundant?